Pluto Data

**Goal**  
Clean and explore the PLUTO tax lot data so it is ready to link to restaurants and to describe neighborhood property characteristics.

**Plan**
1. Load the PLUTO CSV from `data/raw`.
2. Keep only the columns we need 
3. Convert numeric fields (assessed value, year built) to the right types and handle missing.
4. Check basic distributions for borough, land use, building class, assessed value, and year built.
5. Filter or flag obvious outliers or unusable rows if needed.
6. Save a cleaned version 
7. Write a short summary 

In [ ]:
import pandas as pd

pluto_path = "../data/raw/Primary_Land_Use_Tax_Lot_Output_(PLUTO)_20251202csv"
pluto = pd.read_csv(pluto_path)

In [ ]:
pluto.head(20)